In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
import random
import time
import pandas as pd

In [2]:
pth = r'/mnist.pkl.gz'

In [68]:

def load_data():
    
    '''
    load the MNIST data as a tuple contraining the training dta, evaluation data, and the test data.
    Training data has two entries, one is actual training images, which is a numpy ndarray with 50,000 entries, each entry is a numpy ndarray with 784 (28*28) pixels in a single MNIST image
                                  the other is numpy ndarray containing 50,000 entries. Those entries are just the digit values (0...9) for the corresponding images contained in the first entry of the tuple. 
    The ``validation_data`` and ``test_data`` are similar, except each contains only 10,000 images.
    
     This is a nice data format, but for use in neural networks it's helpful to modify the format of the ``training_data`` a little. That's done in the wrapper function ``load_data_wrapper()``, see
below.
    '''
    
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [69]:
class Network(object):
    
    def __init__(self, sizes):
        
        
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
            
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):

            a = self.sigmoid(np.dot(w, a)+b)

        return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta,test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n_train = len(training_data)

        loss_Tr = [] # loss of training set  use upper case T to denote a set of samples
        loss_Te = [] # loss of test set
        acc_Tr  = [] # accuracy of training set
        acc_Te  = [] # accuracy of test set
    
        f = open('model_ep{}_size{}_mb{}_eta{}.txt'.format(epochs, self.sizes,mini_batch_size, eta), 'w')
        f.write('epoch, loss_tr,loss_te, acc_tr, acc_te \n')
        
        if test_data:

            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n_train, mini_batch_size)]
            
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
                          
            else:
                print("Epoch {} complete".format(j))
            
            # compute the loss for training data set
            loss_tr = [0.5 * np.sum((self.feedforward(x) - y)**2) for (x, y) in training_data]

            loss_Tr.append(sum(loss_tr))
            
            
        
            # write the accuracy to 
            # compute the loss for test data set
            if test_data:
                loss_te = [0.5 * np.sum((self.feedforward(x) - vectorized_result(y))**2) for (x, y ) in test_data]

                loss_Te.append(sum(loss_te))
            # compute the accuracy for training data

            res_tr = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in training_data]
            acc_tr = sum(int(x==y) for (x,y) in res_tr) / n_train
            acc_Tr.append(acc_tr)
            
            # accuracy of test set
            if test_data:

                acc_te = self.evaluate(test_data)/n_test
                
                acc_Te.append(acc_te)
            
            
            f.write("{}, {:10}, {:10}, {:.5}, {:.5}\n".format(j, sum(loss_tr), sum(loss_te),acc_tr, acc_te))
            
        f.close()
        return loss_Tr, loss_Te, acc_Tr, acc_Te
    
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
    
    
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def sigmoid(self, z):
        '''simoid function'''
        return 1/(1+np.exp(-z))

    def sigmoid_prime(self, z):
        '''
        derivative of sigmoid function'''

        return self.sigmoid(z) * (1- self.sigmoid(z))

In [22]:

# def plt_acc(acc_Tr, acc_Te, savepath,option):
def plt_acc(acc_Tr, acc_Te):
    # plot epoch vs acc
    fig, axs = plt.subplots(1,2, sharey=True)
#     epo = np.arange(epcochs).tolist()
    epo = np.arange(len(acc_Tr))
    
    axs[0].plot(epo, acc_Tr, 'r-', marker = '^')
    
    axs[0].set_xlabel('Epoches')
    axs[0].set_title('Training set')
    axs[0].set_ylabel('Acciracy')
    

    axs[1].plot(epo, acc_Te, 'b-', marker = '^')
    axs[1].set_xlabel('Epoches')
    axs[1].set_title('Test set')
    axs[1].set_ylabel('Acciracy')
    
#     if savepath:
        
#         plt.savefig('../figures/acc_ep={}_hl={}_lr={}.jpg'.format(option['epoches'], option['hiddenlayer'], option['learningrate']))
        
    return fig 

In [20]:
# def plt_loss(loss_Tr, loss_Te, savepath,option):
def plt_loss(loss_Tr, loss_Te, epochs):
    # plot convergence
    # plot epoch vs acc
    fig, axs = plt.subplots(1,2, sharey=True)
    epo = np.arange(epcochs).tolist()
    
    axs[0].plot(epo, loss_Tr, 'r-', marker = '^')
    
    axs[0].set_xlabel('Epoches')
    axs[0].set_title('Training set')
    axs[0].set_ylabel('Loss')
    

    axs[1].plot(epo, loss_Te, 'b-', marker = '^')
    axs[1].set_xlabel('Epoches')
    axs[1].set_title('Test set')
    axs[1].set_ylabel('Loss')
    
#     if savepath:

#         plt.savefig('../figures/acc_ep={}_hl={}_lr={}.jpg'.format(option['epoches'], option['hiddenlayer'], option['learningrate']))
    
    return fig
    

In [9]:
model_1 = {}
model_1['hiddenlayer'] = 1
model_1['hiddeneurons'] = 30
model_1['learningrate'] = 3
model_1['min_batch'] = 10
model_1['epoches'] = 50 


In [9]:
# base modl: 1 hidden layer, 30 hidden nodes and 1 = 3.0
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, model_1['hiddeneurons'], 10])
loss_Tr_1, loss_Te_1, acc_Tr_1, acc_Te_1 = net.SGD(training_data, model_1['epoches'], model_1['min_batch'], model_1['learningrate'], test_data = test_data)
print('Run time: ',time.time() - t1)

Epoch 0: 8994 / 10000
Epoch 1: 9173 / 10000
Epoch 2: 9261 / 10000
Epoch 3: 9336 / 10000
Epoch 4: 9316 / 10000
Epoch 5: 9353 / 10000
Epoch 6: 9405 / 10000
Epoch 7: 9412 / 10000
Epoch 8: 9421 / 10000
Epoch 9: 9444 / 10000
Epoch 10: 9438 / 10000
Epoch 11: 9456 / 10000
Epoch 12: 9459 / 10000
Epoch 13: 9467 / 10000
Epoch 14: 9467 / 10000
Epoch 15: 9445 / 10000
Epoch 16: 9460 / 10000
Epoch 17: 9465 / 10000
Epoch 18: 9484 / 10000
Epoch 19: 9431 / 10000
Epoch 20: 9458 / 10000
Epoch 21: 9490 / 10000
Epoch 22: 9483 / 10000
Epoch 23: 9486 / 10000
Epoch 24: 9471 / 10000
Epoch 25: 9468 / 10000
Epoch 26: 9468 / 10000
Epoch 27: 9483 / 10000
Epoch 28: 9475 / 10000
Epoch 29: 9487 / 10000
Epoch 30: 9474 / 10000
Epoch 31: 9466 / 10000
Epoch 32: 9487 / 10000
Epoch 33: 9489 / 10000
Epoch 34: 9448 / 10000
Epoch 35: 9469 / 10000
Epoch 36: 9475 / 10000
Epoch 37: 9494 / 10000
Epoch 38: 9479 / 10000
Epoch 39: 9474 / 10000
Epoch 40: 9478 / 10000
Epoch 41: 9478 / 10000
Epoch 42: 9494 / 10000
Epoch 43: 9468 / 1000

In [ ]:
savepath = True
plt_acc(acc_Tr_1, acc_Te_1, savepath, model_1)

In [10]:
model_2 = {}
model_2['hiddenlayer'] = 2
model_2['hiddeneurons'] = 15
model_2['learningrate'] = 3
model_2['min_batch'] = 10
model_2['epoches'] = 50

In [11]:
# model 2: 2 hidden layer, 30 hidden nodes and 1 = 3.0
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, model_2['hiddeneurons'],model_2['hiddeneurons'], 10])
loss_Tr_2, loss_Te_2, acc_Tr_2, acc_Te_2 = net.SGD(training_data, model_2['epoches'], model_2['min_batch'], model_2['learningrate'], test_data = test_data)

Epoch 0: 8945 / 10000
Epoch 1: 9031 / 10000
Epoch 2: 9138 / 10000
Epoch 3: 9169 / 10000
Epoch 4: 9185 / 10000
Epoch 5: 9246 / 10000
Epoch 6: 9191 / 10000
Epoch 7: 9190 / 10000
Epoch 8: 9224 / 10000
Epoch 9: 9288 / 10000
Epoch 10: 9301 / 10000
Epoch 11: 9325 / 10000
Epoch 12: 9229 / 10000
Epoch 13: 9279 / 10000
Epoch 14: 9272 / 10000
Epoch 15: 9308 / 10000
Epoch 16: 9326 / 10000
Epoch 17: 9283 / 10000
Epoch 18: 9347 / 10000
Epoch 19: 9337 / 10000
Epoch 20: 9311 / 10000
Epoch 21: 9310 / 10000
Epoch 22: 9298 / 10000
Epoch 23: 9299 / 10000
Epoch 24: 9309 / 10000
Epoch 25: 9321 / 10000
Epoch 26: 9357 / 10000
Epoch 27: 9347 / 10000
Epoch 28: 9317 / 10000
Epoch 29: 9355 / 10000
Epoch 30: 9298 / 10000
Epoch 31: 9379 / 10000
Epoch 32: 9345 / 10000
Epoch 33: 9351 / 10000
Epoch 34: 9345 / 10000
Epoch 35: 9345 / 10000
Epoch 36: 9331 / 10000
Epoch 37: 9321 / 10000
Epoch 38: 9356 / 10000
Epoch 39: 9368 / 10000
Epoch 40: 9335 / 10000
Epoch 41: 9292 / 10000
Epoch 42: 9374 / 10000
Epoch 43: 9332 / 1000

In [12]:
model_3 = {}
model_3['hiddenlayer'] = 1
model_3['hiddeneurons'] = 15
model_3['learningrate'] = 3
model_3['min_batch'] = 10
model_3['epoches'] = 50

In [13]:
# model 3: 1 hidden layer with 15 neurons, 30 hidden nodes and 1 = 3.0
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, model_3['hiddeneurons'], 10])
loss_Tr_3, loss_Te_3, acc_Tr_3, acc_Te_3 = net.SGD(training_data,   model_3['epoches'], model_3['min_batch'], model_3['learningrate'], test_data = test_data)

Epoch 0: 8926 / 10000
Epoch 1: 9026 / 10000
Epoch 2: 9078 / 10000
Epoch 3: 9120 / 10000
Epoch 4: 9141 / 10000
Epoch 5: 9148 / 10000
Epoch 6: 9171 / 10000
Epoch 7: 9257 / 10000
Epoch 8: 9233 / 10000
Epoch 9: 9232 / 10000
Epoch 10: 9268 / 10000
Epoch 11: 9133 / 10000
Epoch 12: 9244 / 10000
Epoch 13: 9233 / 10000
Epoch 14: 9283 / 10000
Epoch 15: 9304 / 10000
Epoch 16: 9301 / 10000
Epoch 17: 9248 / 10000
Epoch 18: 9300 / 10000
Epoch 19: 9240 / 10000
Epoch 20: 9290 / 10000
Epoch 21: 9305 / 10000
Epoch 22: 9305 / 10000
Epoch 23: 9311 / 10000
Epoch 24: 9295 / 10000
Epoch 25: 9330 / 10000
Epoch 26: 9296 / 10000
Epoch 27: 9307 / 10000
Epoch 28: 9305 / 10000
Epoch 29: 9279 / 10000
Epoch 30: 9253 / 10000
Epoch 31: 9273 / 10000
Epoch 32: 9315 / 10000
Epoch 33: 9322 / 10000
Epoch 34: 9298 / 10000
Epoch 35: 9315 / 10000
Epoch 36: 9333 / 10000
Epoch 37: 9314 / 10000
Epoch 38: 9296 / 10000
Epoch 39: 9334 / 10000
Epoch 40: 9311 / 10000
Epoch 41: 9309 / 10000
Epoch 42: 9312 / 10000
Epoch 43: 9299 / 1000

In [14]:
model_4 = {}
model_4['hiddenlayer'] = 1
model_4['hiddeneurons'] = 30
model_4['learningrate'] = 1
model_4['min_batch'] = 10
model_4['epoches'] = 50

In [15]:
# model 4: 1 hidden layer with 30 neurons and 1 = 1.0
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, model_4['hiddeneurons'], 10])
loss_Tr, loss_Te, acc_Tr, acc_Te = net.SGD(training_data,  model_4['epoches'], model_4['min_batch'], model_4['learningrate'], test_data = test_data)

Epoch 0: 8159 / 10000
Epoch 1: 8980 / 10000
Epoch 2: 9102 / 10000
Epoch 3: 9178 / 10000
Epoch 4: 9202 / 10000
Epoch 5: 9252 / 10000
Epoch 6: 9278 / 10000
Epoch 7: 9273 / 10000
Epoch 8: 9318 / 10000
Epoch 9: 9325 / 10000
Epoch 10: 9298 / 10000
Epoch 11: 9323 / 10000
Epoch 12: 9331 / 10000
Epoch 13: 9345 / 10000
Epoch 14: 9342 / 10000
Epoch 15: 9344 / 10000
Epoch 16: 9372 / 10000
Epoch 17: 9352 / 10000
Epoch 18: 9373 / 10000
Epoch 19: 9377 / 10000
Epoch 20: 9390 / 10000
Epoch 21: 9375 / 10000
Epoch 22: 9400 / 10000
Epoch 23: 9363 / 10000
Epoch 24: 9388 / 10000
Epoch 25: 9377 / 10000
Epoch 26: 9384 / 10000
Epoch 27: 9404 / 10000
Epoch 28: 9375 / 10000
Epoch 29: 9402 / 10000
Epoch 30: 9388 / 10000
Epoch 31: 9398 / 10000
Epoch 32: 9403 / 10000
Epoch 33: 9398 / 10000
Epoch 34: 9410 / 10000
Epoch 35: 9402 / 10000
Epoch 36: 9413 / 10000
Epoch 37: 9408 / 10000
Epoch 38: 9397 / 10000
Epoch 39: 9404 / 10000
Epoch 40: 9407 / 10000
Epoch 41: 9401 / 10000
Epoch 42: 9413 / 10000
Epoch 43: 9416 / 1000

In [16]:
model_5 = {}
model_5['hiddenlayer'] = 1
model_5['hiddeneurons'] = 30
model_5['learningrate'] = 100
model_5['min_batch'] = 10
model_5['epoches'] = 50

In [17]:
# model 5: 1 hidden layer with 30 neurons and 1 = 20.0
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, model_5['hiddeneurons'] , 10])
loss_Tr, loss_Te, acc_Tr, acc_Te = net.SGD(training_data, model_5['epoches'], model_5['min_batch'],model_5['learningrate'] , test_data = test_data)

Epoch 0: 988 / 10000
Epoch 1: 988 / 10000
Epoch 2: 988 / 10000
Epoch 3: 988 / 10000
Epoch 4: 988 / 10000
Epoch 5: 988 / 10000
Epoch 6: 988 / 10000
Epoch 7: 988 / 10000
Epoch 8: 988 / 10000
Epoch 9: 988 / 10000
Epoch 10: 988 / 10000
Epoch 11: 988 / 10000
Epoch 12: 988 / 10000
Epoch 13: 988 / 10000
Epoch 14: 988 / 10000
Epoch 15: 988 / 10000
Epoch 16: 988 / 10000
Epoch 17: 988 / 10000
Epoch 18: 988 / 10000
Epoch 19: 988 / 10000
Epoch 20: 988 / 10000
Epoch 21: 988 / 10000
Epoch 22: 988 / 10000
Epoch 23: 988 / 10000
Epoch 24: 988 / 10000
Epoch 25: 988 / 10000
Epoch 26: 988 / 10000
Epoch 27: 988 / 10000
Epoch 28: 988 / 10000
Epoch 29: 988 / 10000
Epoch 30: 988 / 10000
Epoch 31: 988 / 10000
Epoch 32: 988 / 10000
Epoch 33: 988 / 10000
Epoch 34: 988 / 10000
Epoch 35: 988 / 10000
Epoch 36: 988 / 10000
Epoch 37: 988 / 10000
Epoch 38: 988 / 10000
Epoch 39: 988 / 10000
Epoch 40: 988 / 10000
Epoch 41: 988 / 10000
Epoch 42: 988 / 10000
Epoch 43: 988 / 10000
Epoch 44: 988 / 10000
Epoch 45: 988 / 1000

In [58]:
df = df_1.copy()
size = [784, 30, 10]
learning_rate = 3
epochs = 50

%matplotlib qt
fig, axs = plt.subplots(1,2, sharey=False)

# plot accuracy for test set
axs[0].plot(df.epoch, df.acc_te, 'r-', marker = '^')
axs[0].set_ylim([0.8,1])

axs[0].set_xlabel('Epoch',fontsize = 12)
# axs[0].set_title('Accuracy')
axs[0].set_ylabel('Accuracy',fontsize = 12)

# plot loss for test set
axs[1].plot(df.epoch, df.loss_te, 'b-', marker = '^')
axs[1].set_xlabel('Epoch',fontsize = 12)
# axs[1].set_title('Loss')
axs[1].set_ylabel('Loss',fontsize = 12)
axs[1].set_ylim([400,1400])
fig.suptitle('Model {}, learning_rate = {}, epochs = {}'.format(size, learning_rate, epochs), x= 0.5, y = 1,fontsize = 15)
plt.tight_layout()


In [10]:
# import txt file for ploting accuracy vs. epoch

In [56]:
txt_model_1 = r'./model_ep50_hl30_mb10_eta3.txt'

In [57]:
df_1 = pd.read_csv(txt_model_1, delimiter=',')
df_1.columns = df_1.columns.str.replace(' ', '')
df_1.head()


,epoch,loss_tr,loss_te,acc_tr,acc_te
0,0,4092.376230,818.351089,0.89876,0.8994
1,1,3217.054776,678.137535,0.92162,0.9173
2,2,2862.139399,609.115845,0.93232,0.9261
3,3,2502.114123,548.848547,0.93988,0.9336
4,4,2576.086182,569.742619,0.93854,0.9316


In [71]:
def vectorized_y(j):
    # return a vector of binary result
    e = np.zeros((2, 1))
    e[j] = 1.0
    
    return e

In [70]:
def XOR_data():
    
    tr_x = np.array([[0,0],[1,0],[0,1], [1,1]])
    tr_y = np.array([0,1,1,0])
    tr_input = [np.reshape(x, (2,1)) for x in tr_x]
    tr_output = [vectorized_y(y) for y in tr_y]
    
    tr_data = list(zip(tr_input, tr_output))
    
    return tr_data

In [65]:
# train XOR use network 


array([[0.],
       [1.]])

In [75]:
'a' in 'abd'

True